In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import dump
import pathlib
from joblib import load


2023-12-13 19:26:59.153875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
starting_time_step = datetime(year=2020, month=1, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 900, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
dformer = DataTransformer(env_config=env_config)
dfs = dformer.create_input_dataframes()
market = MarketSimulator(env_config=env_config, data_frames=dfs)
market.reset()
print("here", market.the_current_time_step, market.shares_data)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:279: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:280: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')


here 2020-01-01 00:00:00+00:00 {1: 5319.148936170213, 2: 1188.3541295306002, 3: 116.795141322121, 4: 2.6666666666666665, 5: 776.0962359332557, 6: 103.58937173046046, 7: 215.21575379317764, 8: 1705.0298380221652, 9: 57.29181586410382, 10: 186.7936863734006, 11: 159.4514868851152, 12: 259.36973155232783, 13: 829.8755186721992, 14: 58.394160583941606, 15: 83.07717869901138, 16: 25.38135485672225, 17: 2.6954177897574123, 18: 113.63636363636364, 19: 631.9115323854661, 20: 8.62065249718751, 21: 2114.164904862579, 22: 1509.433962264151, 23: 102.30179028132993, 24: 20.2020202020202, 25: 144.6445360526506, 26: 43.26850269346429, 27: 1777.7777777777778, 28: 394.86673247778873, 29: 85.94387864724335, 30: 1428.5714285714287, 31: 708.4661707403471, 32: 49.5000495000495, 33: 139.9384270920795, 34: 185.8045336306206, 35: 2.4242424242424243, 36: 135.74046423238767, 37: 90.45680687471733, 38: 449.94375703037116, 39: 29.62962962962963, 40: 478.24007651841225, 41: 28.27214769369955, 42: 207.6196408180214

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:309: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:310: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='bfill')


In [3]:
nw_starting_time_step = datetime(year=2023, month=9, day=1)

nw_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nw_env_config = {
    "db_params": nw_db_params, 
    "max_episode_steps": 60, 
    "the_current_time_step": nw_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 60,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nw_dformer = DataTransformer(env_config=nw_env_config)
nw_dfs = nw_dformer.create_input_dataframes()
# display(nw_dfs[2])
nw_market = MarketSimulator(env_config=nw_env_config, data_frames=nw_dfs)
nw_market.reset()
nw_forest = Forester(market=nw_market)
print(nw_market.the_current_time_step, nw_market.shares_data)


/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:279: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:280: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')


2023-09-01 00:00:00+00:00 {1: 7142.857142857143, 2: 883.3922261484099, 3: 141.1930815390046, 4: 0.9409550693954364, 5: 227.50540325332727, 6: 555.7099194220616, 7: 250.53238131028436, 8: 5797.101449275362, 9: 95.48818333731201, 10: 252.17500945656286, 11: 46.53760238272524, 12: 484.26150121065376, 13: 700.280112044818, 14: 163.9344262295082, 15: 150.71590052750565, 16: 62.5, 17: 2.5, 18: 174.97812773403325, 19: 244.76808224207565, 20: 28.78526194588371, 21: 3412.9692832764504, 22: 2597.402597402597, 23: 180.1477211313277, 24: 30.721966205837173, 25: 295.90176061547567, 26: 44.8058785312633, 27: 5571.030640668524, 28: 401.3646397752358, 29: 188.96447467876038, 30: 4819.277108433735, 31: 2183.406113537118, 32: 57.12816704276044, 33: 211.72983273343215, 34: 339.7893306150187, 35: 2.857142857142857, 36: 104.8767697954903, 37: 188.78610534264678, 38: 942.0631182289213, 39: 68.96551724137932, 40: 418.4975936388366, 41: 45.236587351850176, 42: 172.7264876068745, 43: 153.25670498084293, 44: 12

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:309: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:310: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[field].fillna(method='bfill')


In [ ]:
pd.set_option('display.max_rows', None)
display(nw_dfs[2])

In [4]:
forest = Forester(market=market)
forest.make_forest_data()
X_train = forest.train_input
y_train = forest.train_output
y_test = forest.test_output
X_test = forest.test_input
print(forest.market.the_current_time_step)

nw_forest.make_forest_data()
nw_X_train = nw_forest.train_input
nw_y_train = nw_forest.train_output
nw_y_test = nw_forest.test_output
nw_X_test = nw_forest.test_input
print(nw_forest.market.the_current_time_step)

2023-06-14 00:00:00+00:00
2023-11-24 00:00:00+00:00


In [6]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [73]
# max_depth = [int(x) for x in np.linspace(10, 80, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5]
# min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

best_params = {
    'n_estimators': None,
    'max_features': None,
    'max_depth': None,
    'min_samples_split': None,
    'min_samples_leaf': None,
    'bootstrap': None,
    "accuracy": None
}

best_accuracy = 0

In [7]:
iters = 1
for n_ests in n_estimators:
    for max_feat in max_features:
        for max_dept in max_depth:
            for min_sample_sp in min_samples_split:
                for min_sample_lf in min_samples_leaf:
                    for bt_strp in bootstrap:
                        rf = RandomForestClassifier(
                            n_estimators=n_ests,
                            max_features=max_feat,
                            max_depth=max_dept,
                            min_samples_split=min_sample_sp,
                            min_samples_leaf=min_sample_lf,
                            bootstrap=bt_strp
                        )
                        rf.fit(X_train, y_train)
                        nw_X_test = nw_forest.train_input
                        nw_y_pred = rf.predict(nw_X_test)

                        nw_y_test = nw_forest.train_output
                        nw_y_test_first_column = [row[1] for row in nw_y_test]
                        nw_y_pred_first_column = [row[1] for row in nw_y_pred]
                        accuracy = accuracy_score(nw_y_test_first_column, nw_y_pred_first_column)
                        print(f"{iters} Accuracy:", accuracy)
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params["n_estimators"]=n_ests
                            best_params["max_features"]=max_feat
                            best_params["max_depth"]=max_dept
                            best_params["min_samples_split"]=min_sample_sp
                            best_params["min_samples_leaf"]=min_sample_lf
                            best_params["bootstrap"]=bt_strp
                            best_params["accuracy"] = accuracy
                        iters = iters + 1

KeyboardInterrupt: 

In [8]:
print(best_params)

{'n_estimators': None, 'max_features': None, 'max_depth': None, 'min_samples_split': None, 'min_samples_leaf': None, 'bootstrap': None, 'accuracy': None}


In [17]:
rf = RandomForestClassifier(
    n_estimators=best_params["n_estimators"],
    max_features=best_params["max_features"],
    max_depth=best_params["max_depth"],
    min_samples_split=best_params["min_samples_split"],
    min_samples_leaf=best_params["min_samples_leaf"],
    bootstrap=best_params["bootstrap"]
)

(current_file_path, new_file_path) = DataTransformer.get_new_file_path()
rf.fit(X_train, y_train)  # Your existing code

dump(rf, new_file_path)

['/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/joblibs/trading_job_v1']

In [12]:
(current_file_path, new_file_path) = DataTransformer.get_new_file_path()

rf = load(pathlib.Path(current_file_path))
nwa_starting_time_step = datetime(year=2023, month=1, day=1)

nwa_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwa_env_config = {
    "db_params": nwa_db_params, 
    "max_episode_steps": 100, 
    "the_current_time_step": nwa_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nwa_dformer = DataTransformer(env_config=nw_env_config)
nwa_dfs = nwa_dformer.create_input_dataframes()
# display(nwa_dfs[2])
nwa_market = MarketSimulator(env_config=nw_env_config, data_frames=nwa_dfs)
nwa_market.reset()
nwa_forest = Forester(market=nwa_market)

nwa_forest.make_forest_data()

nwa_X_train = nwa_forest.train_input
nwa_y_train = nwa_forest.train_output

nwa_y_pred = rf.predict(nwa_X_train)

nwa_y_test = nwa_forest.train_output
nwa_y_test_first_column = [row[0] for row in nwa_y_test]
nwa_y_pred_first_column = [row[0] for row in nwa_y_pred]

accuracy = accuracy_score(nwa_y_test_first_column, nwa_y_pred_first_column)
print("Accuracy:", accuracy)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:275: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:276: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:305: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

Accuracy: 0.18867924528301888


In [15]:
nwb_starting_time_step = datetime(year=2023, month=1, day=1)

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 100, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
# display(nwb_dfs[2])
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step)
for indx in range(100):
    if indx < 7:
        acts = np.full(100, 15)
        rand_acts = nwa_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        flat_state = np.array([nwa_forest.get_flattened_states(nwb_market.state)])
        my_pred = rf.predict(flat_state)
        acts = np.array([15,15,15,15,15])
        # acts = my_pred[0]
        rand_acts = nwa_forest.act_random(acts)
        nwb_market.step(rand_acts)

print(nwb_market.wallet_state)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:275: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:276: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:305: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

2100000.0
2023-01-02 00:00:00+00:00
0.0


In [16]:
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))
final_total_potfolio = total_potfolio
gain = final_total_potfolio - initial_total_potfolio
print("gain", gain)

1947374.2297988413
2023-05-17 20:00:00+00:00
gain -152625.7702011587
